In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a Convolutional Block (no residuals)
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

# Define an Upsampling Block
class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpConv, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        
    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return x

# Define the UNet Model
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = ConvBlock(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = ConvBlock(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = ConvBlock(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = ConvBlock(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        
        # Bridge
        self.bridge = ConvBlock(512, 1024)
        
        # Decoder
        self.dec4 = UpConv(1024, 512)
        self.dec4_conv = ConvBlock(1024, 512)
        self.dec3 = UpConv(512, 256)
        self.dec3_conv = ConvBlock(512, 256)
        self.dec2 = UpConv(256, 128)
        self.dec2_conv = ConvBlock(256, 128)
        self.dec1 = UpConv(128, 64)
        self.dec1_conv = ConvBlock(128, 64)
        
        # Output
        self.final = nn.Sequential(
            nn.Conv2d(64, out_channels, kernel_size=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool1(enc1))
        enc3 = self.enc3(self.pool2(enc2))
        enc4 = self.enc4(self.pool3(enc3))
        
        # Bridge
        bridge = self.bridge(self.pool4(enc4))
        
        # Decoder
        dec4 = self.dec4(bridge, enc4)
        dec4 = self.dec4_conv(dec4)
        dec3 = self.dec3(dec4, enc3)
        dec3 = self.dec3_conv(dec3)
        dec2 = self.dec2(dec3, enc2)
        dec2 = self.dec2_conv(dec2)
        dec1 = self.dec1(dec2, enc1)
        dec1 = self.dec1_conv(dec1)
        
        # Output
        return self.final(dec1)


In [4]:
from segmentation import run_segmentation_pipeline

# Create your model instance
my_model = UNet(in_channels=1, out_channels=1)

# Run the pipeline with custom model and naming
trained_model, dice_score = run_segmentation_pipeline(
    model=my_model,
    base_dir="kaggle_3m",
    model_name="Base_Unet",
    batch_size=8,
    patience=10,
    epochs=100
)

Using device: cuda
Loading data...


 18%|█▊        | 20/110 [00:00<00:01, 54.21it/s]


KeyboardInterrupt: 